In [2]:
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_features.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/submission_format.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_labels.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/test_set_features.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


## Importing the needed libraries

In [203]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  f1_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier



%matplotlib inline


## Importing data

In [6]:
path_train_X='./training_set_features.csv'
path_train_Y='./training_set_labels.csv'
path_test_X='./test_set_features.csv'

In [127]:
train_set = pd.read_csv(path_train_X)
print(train_set.head())

   respondent_id  h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  \
0              0           1.0             0.0                        0.0   
1              1           3.0             2.0                        0.0   
2              2           1.0             1.0                        0.0   
3              3           1.0             1.0                        0.0   
4              4           2.0             1.0                        0.0   

   behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  \
0                   0.0                   0.0                    0.0   
1                   1.0                   0.0                    1.0   
2                   1.0                   0.0                    0.0   
3                   1.0                   0.0                    1.0   
4                   1.0                   0.0                    1.0   

   behavioral_large_gatherings  behavioral_outside_home  \
0                          0.0               

In [128]:
train_set_target = pd.read_csv(path_train_Y)
print(train_set_target.head())
print(train_set_target.shape)

   respondent_id  h1n1_vaccine  seasonal_vaccine
0              0             0                 0
1              1             0                 1
2              2             0                 0
3              3             0                 1
4              4             0                 0
(26707, 3)


In [129]:
test_set_X = pd.read_csv(path_test_X)
test_set_X.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [130]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
respondent_id                  26707 non-null int64
h1n1_concern                   26615 non-null float64
h1n1_knowledge                 26591 non-null float64
behavioral_antiviral_meds      26636 non-null float64
behavioral_avoidance           26499 non-null float64
behavioral_face_mask           26688 non-null float64
behavioral_wash_hands          26665 non-null float64
behavioral_large_gatherings    26620 non-null float64
behavioral_outside_home        26625 non-null float64
behavioral_touch_face          26579 non-null float64
doctor_recc_h1n1               24547 non-null float64
doctor_recc_seasonal           24547 non-null float64
chronic_med_condition          25736 non-null float64
child_under_6_months           25887 non-null float64
health_worker                  25903 non-null float64
health_insurance               14433 non-null float64
opinion_h1n1_vacc_effective  

In [131]:
test_set_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26708 entries, 0 to 26707
Data columns (total 36 columns):
respondent_id                  26708 non-null int64
h1n1_concern                   26623 non-null float64
h1n1_knowledge                 26586 non-null float64
behavioral_antiviral_meds      26629 non-null float64
behavioral_avoidance           26495 non-null float64
behavioral_face_mask           26689 non-null float64
behavioral_wash_hands          26668 non-null float64
behavioral_large_gatherings    26636 non-null float64
behavioral_outside_home        26626 non-null float64
behavioral_touch_face          26580 non-null float64
doctor_recc_h1n1               24548 non-null float64
doctor_recc_seasonal           24548 non-null float64
chronic_med_condition          25776 non-null float64
child_under_6_months           25895 non-null float64
health_worker                  25919 non-null float64
health_insurance               14480 non-null float64
opinion_h1n1_vacc_effective  

## Dealing with null values:

In [132]:
train_set.isnull().sum()

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

### Deleting columns with too mant missing values:

In [133]:
sums = train_set.isnull().sum()
columns = list(sums[sums > 0.4 * train_set.shape[0]].keys())
columns

['health_insurance', 'employment_industry', 'employment_occupation']

In [134]:
df = train_set.drop(columns=columns)
df = df.drop(columns=['respondent_id',])
df.isnull().sum()[:5]

h1n1_concern                  92
h1n1_knowledge               116
behavioral_antiviral_meds     71
behavioral_avoidance         208
behavioral_face_mask          19
dtype: int64

In [135]:
df.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0


### Imputing the others with the most frequent value:

In [136]:
impt = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
df = pd.DataFrame(impt.fit_transform(df),columns=df.columns)
df.isnull().sum().sum()

0

In [137]:
df.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children
0,1,0,0,0,0,0,0,1,1,0,...,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0,0
1,3,2,0,1,0,1,0,1,1,0,...,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0,0
2,1,1,0,1,0,0,0,0,0,0,...,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2,0
3,1,1,0,1,0,1,1,0,0,0,...,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0,0
4,2,1,0,1,0,1,1,0,1,0,...,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1,0


## using LabelEncoder to label the columns:

In [149]:
le = LabelEncoder()
classes= {}
for col in df.columns:
    df[col] = le.fit_transform(df[col])
    classes[col] = le.classes_
classes    

{'h1n1_concern': array([0, 1, 2, 3]),
 'h1n1_knowledge': array([0, 1, 2]),
 'behavioral_antiviral_meds': array([0, 1]),
 'behavioral_avoidance': array([0, 1]),
 'behavioral_face_mask': array([0, 1]),
 'behavioral_wash_hands': array([0, 1]),
 'behavioral_large_gatherings': array([0, 1]),
 'behavioral_outside_home': array([0, 1]),
 'behavioral_touch_face': array([0, 1]),
 'doctor_recc_h1n1': array([0, 1]),
 'doctor_recc_seasonal': array([0, 1]),
 'chronic_med_condition': array([0, 1]),
 'child_under_6_months': array([0, 1]),
 'health_worker': array([0, 1]),
 'opinion_h1n1_vacc_effective': array([0, 1, 2, 3, 4]),
 'opinion_h1n1_risk': array([0, 1, 2, 3, 4]),
 'opinion_h1n1_sick_from_vacc': array([0, 1, 2, 3, 4]),
 'opinion_seas_vacc_effective': array([0, 1, 2, 3, 4]),
 'opinion_seas_risk': array([0, 1, 2, 3, 4]),
 'opinion_seas_sick_from_vacc': array([0, 1, 2, 3, 4]),
 'age_group': array([0, 1, 2, 3, 4]),
 'education': array([0, 1, 2, 3]),
 'race': array([0, 1, 2, 3]),
 'sex': array([0, 1

## Extracting non-binary features:

In [150]:
columns_categorical=[]
for col in df.columns[1:]:
    if len(df[col].unique()) > 2:
        columns_categorical.append(col)
columns_categorical

['h1n1_knowledge',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'income_poverty',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'household_adults',
 'household_children']

## OHE for the non-binary columns

In [261]:
X = pd.get_dummies(df,columns=columns_categorical)
X.head()

,h1n1_concern,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,...,census_msa_1,census_msa_2,household_adults_0,household_adults_1,household_adults_2,household_adults_3,household_children_0,household_children_1,household_children_2,household_children_3
0,1,0,0,0,0,0,1,1,0,0,...,0,1,1,0,0,0,1,0,0,0
1,3,0,1,0,1,0,1,1,0,0,...,0,0,1,0,0,0,1,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,1,0,1,0,1,1,0,0,0,1,...,1,0,1,0,0,0,1,0,0,0
4,2,0,1,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0


## Feature Extraction using PCA:

In [262]:
pca = PCA()
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [263]:
num = (pca.explained_variance_ratio_*100).astype(int).sum()
num

62

In [264]:
pca = PCA(n_components=num)
X_reduced = pca.fit_transform(X)

In [265]:
X_reduced[:5,:5]

array([[ 0.77670033,  1.4469025 , -0.4924783 ,  0.33675145,  0.13754647],
       [-1.50706072, -0.2320853 , -0.15733297,  1.12003893,  0.8845385 ],
       [ 1.5555455 ,  0.16823956, -0.29750636,  0.75661616,  0.03583162],
       [-0.08439037,  2.11925911,  0.18912767,  0.09258918,  0.20320402],
       [-0.29770435, -0.29034152, -0.29825565,  0.24371332, -0.81702476]])

## First dealing with seasonal vaccine

In [266]:
train_set_target.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [267]:
Y_seasonal = train_set_target['seasonal_vaccine'].to_numpy().ravel()

## Applying Train/Valid/Test split

In [268]:
X_train,X_valid,Y_train,Y_valid = train_test_split(X_reduced,Y_seasonal,test_size=.1,random_state=102,stratify=Y_seasonal)

## Applying a logistic regression model:

In [269]:
logreg = LogisticRegression(max_iter=1001,random_state=101,solver='liblinear',penalty='l1')
logreg.fit(X_train,Y_train)
logreg.score(X_train,Y_train)*100, '%'

(77.97886503577966, '%')

In [270]:
print(f1_score(logreg.predict(X_valid),Y_valid)*100,'%')


76.43724696356276 %


In [271]:
pd.DataFrame(classification_report(logreg.predict(X_valid),Y_valid,output_dict = True)).T*100

,f1-score,precision,recall,support
0,79.735376,80.238262,79.238754,144500.0
1,76.437247,75.884244,76.998369,122600.0
micro avg,78.210408,78.210408,78.210408,267100.0
macro avg,78.086312,78.061253,78.118562,267100.0
weighted avg,78.221521,78.239750,78.210408,267100.0


## Only keeping the important columns:

In [287]:
mask = abs(logreg.coef_ )  > 1e-5
mask = mask.ravel()
mask.sum()

61

In [288]:
X_reduced = X_reduced[:,mask]
X_train,X_valid,Y_train,Y_valid = train_test_split(X_reduced,Y_seasonal,test_size=.1,random_state=102,stratify=Y_seasonal)
X_train.shape[1]

61

In [291]:
logreg = LogisticRegression(max_iter=1001,random_state=101,solver='liblinear',penalty='l1')
logreg.fit(X_train,Y_train)
'{:.2f}'.formalogreg.score(X_train,Y_train)*100 , f1_score(logreg.predict(X_valid),Y_valid)*100,'%'

(0.008865035779661039, 76.43724696356276, '%')